In [1]:
import torch
import torchvision.transforms as tvt
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import ConcatDataset, DataLoader
from torchmetrics import AUROC  # additional dependency
from torchvision.datasets import MNIST, FashionMNIST, KMNIST

from pytorch_ood import NegativeEnergy, Softmax
from pytorch_ood.dataset.img import MNISTC, UniformNoise, GaussianNoise
from pytorch_ood.model import WideResNet
from pytorch_ood.utils import is_unknown
from pytorch_ood.transforms import ToUnknown

In [2]:
torch.manual_seed(123)

trans = tvt.Compose([tvt.Resize((32,32)), tvt.ToTensor()])

# setup data
dataset_train = MNIST(root="data", train=True, download=True, transform=trans)
train_loader = DataLoader(dataset_train, batch_size=128)

In [3]:
model = WideResNet(num_classes=10, in_channels=1)
opti = Adam(model.parameters())
criterion = CrossEntropyLoss()
model = model.cuda()

In [4]:
model.train()

# start training
for epoch in range(1):
    for n, batch in enumerate(train_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        logits = model(x)
        loss = criterion(logits, y)
        opti.zero_grad()
        loss.backward()
        opti.step()


In [5]:
# create some methods
energy = NegativeEnergy(model).cuda()
softmax = Softmax(model).cuda()

In [6]:
dataset_in_test = MNIST(root="data", train=False, transform=trans)
dataset_out_test = MNISTC(root="data", subset="all", split="test", download=True,
                          transform=trans, target_transform=ToUnknown())
dataset_test = dataset_in_test + dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

# evaluate
auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

/home/ki/.local/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


tensor(0.5623, device='cuda:0')
tensor(0.5471, device='cuda:0')


In [7]:
# dataset_in_test = MNIST(root="data", train=False, transform=trans)
# dataset_out_test = FashionMNIST(root="data", download=True, transform=trans, target_transform=ToUnknown())
# dataset_test = dataset_in_test + dataset_out_test
# test_loader = DataLoader(dataset_test, batch_size=128)
#
# # evaluate
# auroc_energy = AUROC(num_classes=2)
# auroc_softmax = AUROC(num_classes=2)
# model.eval()
#
# with torch.no_grad():
#     for n, batch in enumerate(test_loader):
#         x, y = batch
#         x = x.cuda()
#         y = y.cuda()
#
#         auroc_energy.update(energy(x), is_unknown(y))
#         auroc_softmax.update(softmax(x), is_unknown(y))
#
# print(auroc_softmax.compute())
# print(auroc_energy.compute())

In [8]:
from pytorch_ood import Mahalanobis
maha = Mahalanobis(model).cuda()
maha.fit(train_loader)

10
tensor(9)


TypeError: only integer tensors of a single element can be converted to an index